In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)


In [2]:
DATA_PATH = "../data/raw/"

df = pd.read_csv(f"{DATA_PATH}/ethiopia_fi_unified_data.csv")
ref = pd.read_csv(f"{DATA_PATH}/reference_codes.csv")

print("Unified dataset shape:", df.shape)
print("Reference codes shape:", ref.shape)


Unified dataset shape: (43, 34)
Reference codes shape: (71, 4)


In [4]:
df.head(10)


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,unit,observation_date,period_start,period_end,fiscal_year,gender,location,region,source_name,source_type,source_url,confidence,related_indicator,relationship_type,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,REC_0001,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,22.00,NaN,percentage,%,2014-12-31,NaN,NaN,2014,all,national,NaN,Global Findex 2014,survey,https://www.worldbank.org/en/publication/globa...,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Baseline year,NaN
1,REC_0002,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,35.00,NaN,percentage,%,2017-12-31,NaN,NaN,2017,all,national,NaN,Global Findex 2017,survey,https://www.worldbank.org/en/publication/globa...,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
2,REC_0003,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,46.00,NaN,percentage,%,2021-12-31,NaN,NaN,2021,all,national,NaN,Global Findex 2021,survey,https://www.worldbank.org/en/publication/globa...,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
3,REC_0004,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,56.00,NaN,percentage,%,2021-12-31,NaN,NaN,2021,male,national,NaN,Global Findex 2021,survey,https://www.worldbank.org/en/publication/globa...,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN
4,REC_0005,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,36.00,NaN,percentage,%,2021-12-31,NaN,NaN,2021,female,national,NaN,Global Findex 2021,survey,https://www.worldbank.org/en/publication/globa...,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN
5,REC_0006,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,49.00,NaN,percentage,%,2024-11-29,2024-10-15,2024-11-29,2024,all,national,NaN,Global Findex 2024,survey,https://www.worldbank.org/en/publication/globa...,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,Account ownership increased from 46% to 49%,Survey Oct-Nov 2024,NaN
6,REC_0007,observation,NaN,ACCESS,Mobile Money Account Rate,ACC_MM_ACCOUNT,higher_better,4.70,NaN,percentage,%,2021-12-31,NaN,NaN,2021,all,national,NaN,Global Findex 2021,survey,https://www.worldbank.org/en/publication/globa...,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
7,REC_0008,observation,NaN,ACCESS,Mobile Money Account Rate,ACC_MM_ACCOUNT,higher_better,9.45,NaN,percentage,%,2024-11-29,NaN,NaN,2024,all,national,NaN,Global Findex 2024,survey,https://www.worldbank.org/en/publication/globa...,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Doubled from 2021,NaN
8,REC_0009,observation,NaN,ACCESS,4G Population Coverage,ACC_4G_COV,higher_better,37.50,NaN,percentage,%,2023-06-30,NaN,NaN,FY2022/23,all,national,NaN,Ethio Telecom LEAD Report,operator,https://www.ethiotelecom.et/,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Before major expansion,NaN
9,REC_0010,observation,NaN,ACCESS,4G Population Coverage,ACC_4G_COV,higher_better,70.80,NaN,percentage,%,2025-06-30,NaN,NaN,FY2024/25,all,national,NaN,Ethio Telecom LEAD Report,operator,https://www.ethiotelecom.et/,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,4G coverage doubled from 37.5% to 70.8%,Major infrastructure investment,NaN


In [5]:
df.info()


<class 'pandas.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   record_id            43 non-null     str    
 1   record_type          43 non-null     str    
 2   category             10 non-null     str    
 3   pillar               33 non-null     str    
 4   indicator            43 non-null     str    
 5   indicator_code       43 non-null     str    
 6   indicator_direction  33 non-null     str    
 7   value_numeric        33 non-null     float64
 8   value_text           10 non-null     str    
 9   value_type           43 non-null     str    
 10  unit                 33 non-null     str    
 11  observation_date     43 non-null     str    
 12  period_start         10 non-null     str    
 13  period_end           10 non-null     str    
 14  fiscal_year          43 non-null     str    
 15  gender               43 non-null     str    
 16  loc

In [6]:
df["record_type"].value_counts()


record_type
observation    30
event          10
target          3
Name: count, dtype: int64

In [6]:
df.groupby(["record_type", "pillar"]).size().reset_index(name="count")

,record_type,pillar,count
0,observation,ACCESS,14
1,observation,AFFORDABILITY,1
2,observation,GENDER,4
3,observation,USAGE,11
4,target,ACCESS,2
5,target,GENDER,1


In [7]:
df["observation_date"] = pd.to_datetime(df["observation_date"], errors="coerce")

df.groupby("record_type")["observation_date"].agg(["min", "max"])


,min,max
record_type,,
event,2021-05-17,2025-12-18
observation,2014-12-31,2025-12-31
target,2025-12-31,2030-12-31


In [8]:
indicators = (
    df[df["record_type"] == "observation"]
    .groupby("indicator_code")["observation_date"]
    .agg(["count", "min", "max"])
    .sort_values("count", ascending=False)
)

indicators


,count,min,max
indicator_code,,,
ACC_OWNERSHIP,6,2014-12-31,2024-11-29
ACC_FAYDA,3,2024-08-15,2025-05-15
ACC_4G_COV,2,2023-06-30,2025-06-30
ACC_MM_ACCOUNT,2,2021-12-31,2024-11-29
GEN_GAP_ACC,2,2021-12-31,2024-11-29
USG_P2P_COUNT,2,2024-07-07,2025-07-07
ACC_MOBILE_PEN,1,2025-12-31,2025-12-31
GEN_GAP_MOBILE,1,2024-12-31,2024-12-31
GEN_MM_SHARE,1,2024-12-31,2024-12-31


In [11]:
events = df[df["record_type"] == "event"][[
    "record_id",
    "category",
    "indicator",        # event description
    "observation_date",
    "source_name",
    "confidence"
]]

events.sort_values("observation_date")


,record_id,category,indicator,observation_date,source_name,confidence
33,EVT_0001,product_launch,Telebirr Launch,2021-05-17,Ethio Telecom,high
41,EVT_0009,policy,NFIS-II Strategy Launch,2021-09-01,NBE,high
34,EVT_0002,market_entry,Safaricom Ethiopia Commercial Launch,2022-08-01,News,high
35,EVT_0003,product_launch,M-Pesa Ethiopia Launch,2023-08-01,Safaricom,high
36,EVT_0004,infrastructure,Fayda Digital ID Program Rollout,2024-01-01,NIDP,high
37,EVT_0005,policy,Foreign Exchange Liberalization,2024-07-29,NBE,high
38,EVT_0006,milestone,P2P Transaction Count Surpasses ATM,2024-10-01,EthSwitch,high
39,EVT_0007,partnership,M-Pesa EthSwitch Integration,2025-10-27,EthSwitch,high
42,EVT_0010,pricing,Safaricom Ethiopia Price Increase,2025-12-15,News,high
40,EVT_0008,infrastructure,EthioPay Instant Payment System Launch,2025-12-18,NBE/EthSwitch,high


In [10]:
df.columns.tolist()


['record_id',
 'record_type',
 'category',
 'pillar',
 'indicator',
 'indicator_code',
 'indicator_direction',
 'value_numeric',
 'value_text',
 'value_type',
 'unit',
 'observation_date',
 'period_start',
 'period_end',
 'fiscal_year',
 'gender',
 'location',
 'region',
 'source_name',
 'source_type',
 'source_url',
 'confidence',
 'related_indicator',
 'relationship_type',
 'impact_direction',
 'impact_magnitude',
 'impact_estimate',
 'lag_months',
 'evidence_basis',
 'comparable_country',
 'collected_by',
 'collection_date',
 'original_text',
 'notes']

In [12]:
df[df["record_type"] == "observation"]["indicator_code"].value_counts()

indicator_code
ACC_OWNERSHIP         6
ACC_FAYDA             3
ACC_MM_ACCOUNT        2
ACC_4G_COV            2
USG_P2P_COUNT         2
GEN_GAP_ACC           2
ACC_MOBILE_PEN        1
USG_P2P_VALUE         1
USG_ATM_COUNT         1
USG_ATM_VALUE         1
USG_CROSSOVER         1
USG_TELEBIRR_USERS    1
USG_TELEBIRR_VALUE    1
USG_MPESA_USERS       1
USG_MPESA_ACTIVE      1
USG_ACTIVE_RATE       1
AFF_DATA_INCOME       1
GEN_MM_SHARE          1
GEN_GAP_MOBILE        1
Name: count, dtype: int64

In [13]:
df[df["record_type"] == "observation"]["observation_date"].agg(["min", "max"])

min    2014-12-31
max    2025-12-31
Name: observation_date, dtype: str

In [16]:
df[df["record_type"] == "event"][
    [
        "record_id",
        "category",
        "period_start",
        "period_end",
        "notes",
        "original_text",
        "source_name",
        "confidence"
    ]
]


,record_id,category,period_start,period_end,notes,original_text,source_name,confidence
33,EVT_0001,product_launch,NaN,NaN,NaN,First major mobile money service in Ethiopia,Ethio Telecom,high
34,EVT_0002,market_entry,NaN,NaN,NaN,End of state telecom monopoly,News,high
35,EVT_0003,product_launch,NaN,NaN,NaN,Second mobile money entrant,Safaricom,high
36,EVT_0004,infrastructure,NaN,NaN,NaN,National biometric digital ID system,NIDP,high
37,EVT_0005,policy,NaN,NaN,NaN,Birr float introduced,NBE,high
38,EVT_0006,milestone,NaN,NaN,NaN,Historic: digital > cash for first time,EthSwitch,high
39,EVT_0007,partnership,NaN,NaN,NaN,Full interoperability for M-Pesa,EthSwitch,high
40,EVT_0008,infrastructure,NaN,NaN,NaN,National real-time payment system,NBE/EthSwitch,high
41,EVT_0009,policy,2021-09-01,2025-06-30,NaN,5-year national financial inclusion strategy,NBE,high
42,EVT_0010,pricing,NaN,NaN,NaN,Data and voice prices increased 20-82%,News,high


In [15]:
df.columns.tolist()


['record_id',
 'record_type',
 'category',
 'pillar',
 'indicator',
 'indicator_code',
 'indicator_direction',
 'value_numeric',
 'value_text',
 'value_type',
 'unit',
 'observation_date',
 'period_start',
 'period_end',
 'fiscal_year',
 'gender',
 'location',
 'region',
 'source_name',
 'source_type',
 'source_url',
 'confidence',
 'related_indicator',
 'relationship_type',
 'impact_direction',
 'impact_magnitude',
 'impact_estimate',
 'lag_months',
 'evidence_basis',
 'comparable_country',
 'collected_by',
 'collection_date',
 'original_text',
 'notes']

In [13]:
impact_links = df[df["record_type"] == "impact_link"][[
    "pillar",
    "related_indicator",
    "relationship_type",
    "impact_direction",
    "impact_magnitude",
    "impact_estimate",
    "lag_months",
    "evidence_basis",
    "confidence"
]]

impact_links


,pillar,related_indicator,relationship_type,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,confidence


In [14]:
df["confidence"].value_counts(dropna=False)


confidence
high      40
medium     3
Name: count, dtype: int64

In [15]:
df.groupby(["record_type", "confidence"]).size().unstack(fill_value=0)


confidence,high,medium
record_type,,
event,10,0
observation,28,2
target,2,1


In [16]:
obs = df[df["record_type"] == "observation"]

indicator_coverage = (
    obs.groupby("indicator_code")["observation_date"]
    .nunique()
    .sort_values(ascending=False)
)

indicator_coverage


indicator_code
ACC_OWNERSHIP         4
ACC_FAYDA             3
ACC_4G_COV            2
ACC_MM_ACCOUNT        2
GEN_GAP_ACC           2
USG_P2P_COUNT         2
ACC_MOBILE_PEN        1
GEN_GAP_MOBILE        1
GEN_MM_SHARE          1
USG_ACTIVE_RATE       1
AFF_DATA_INCOME       1
USG_ATM_COUNT         1
USG_ATM_VALUE         1
USG_MPESA_ACTIVE      1
USG_CROSSOVER         1
USG_MPESA_USERS       1
USG_P2P_VALUE         1
USG_TELEBIRR_USERS    1
USG_TELEBIRR_VALUE    1
Name: observation_date, dtype: int64

In [17]:
key_indicators = [
    "ACC_OWNERSHIP",
    "USG_DIGITAL_PAYMENT",
    "ACC_MM_ACCOUNT"
]

obs[obs["indicator_code"].isin(key_indicators)] \
    .groupby("indicator_code")["observation_date"] \
    .agg(["count", "min", "max"])


,count,min,max
indicator_code,,,
ACC_MM_ACCOUNT,2,2021-12-31,2024-11-29
ACC_OWNERSHIP,6,2014-12-31,2024-11-29


In [19]:
enriched = pd.read_csv("../data/processed/ethiopia_fi_unified_data_enriched.csv")

enriched[
    (enriched["indicator_code"] == "ENB_SMARTPHONE_PCT")
    & (enriched["record_type"] == "observation")
][["indicator", "value_numeric", "observation_date", "confidence"]]


,indicator,value_numeric,observation_date,confidence
0,Smartphone penetration rate,17,2018-01-01,medium
1,Smartphone penetration rate,23,2020-01-01,medium
2,Smartphone penetration rate,32,2023-01-01,medium


In [20]:
df["pillar"].unique()
df["record_type"].value_counts()

record_type
observation    30
event          10
target          3
Name: count, dtype: int64